# Exercises for "Hands-on with Pydata: How to Build a Minimal Recommendation Engine"

# Systems check: imports and files

In [3]:
import numpy as np
import pandas as pd

# Pandas questions: Series and DataFrames
## 1. Adding a column in a DataFrame

In [4]:
# given the following DataFrame, add a new column to it
df = pd.DataFrame({'col1': [1,2,3,4]})
df

,col1
0,1
1,2
2,3
3,4


## 2. Deleting a row in a DataFrame

In [5]:
# given the following DataFrame, delete row 'd' from it
df = pd.DataFrame({'col1': [1,2,3,4]}, index = ['a','b','c','d'])
df.drop

<bound method NDFrame.drop of    col1
a     1
b     2
c     3
d     4>

## 3. Creating a DataFrame from a few Series

In [6]:
# given the following three Series, create a DataFrame such that it holds them as its columns
ser_1 = pd.Series(np.random.randn(6))
ser_2 = pd.Series(np.random.randn(6))
ser_3 = pd.Series(np.random.randn(6))
pd.DataFrame({'col1':ser_1, 'col2':ser_2, 'col3':ser_3})


,col1,col2,col3
0,-1.903740,0.598167,0.093603
1,0.411175,-1.745618,0.473977
2,-0.597083,0.072635,-1.386144
3,0.198180,-1.669397,1.339991
4,1.803933,0.304626,-1.701647
5,0.461596,-0.379844,0.163153


# Pandas questions: Indexing

## 1. Indexing into a specific column

In [7]:
# given the following DataFrame, try to index into the 'col_2' column
df = pd.DataFrame(data={'col_1': [0.12, 7, 45, 10], 'col_2': [0.9, 9, 34, 11]},
                  columns=['col_1', 'col_2', 'col_3'],
                  index=['obs1', 'obs2', 'obs3', 'obs4'])
df

,col_1,col_2,col_3
obs1,0.12,0.9,NaN
obs2,7.00,9.0,NaN
obs3,45.00,34.0,NaN
obs4,10.00,11.0,NaN


## 2. Label-based indexing

In [8]:
# using the same DataFrame, index into the row whose index is 'obs3'
df.ix['obs3']


col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

In [9]:
df.loc['obs3']

col_1     45
col_2     34
col_3    NaN
Name: obs3, dtype: object

## 2. Position-based indexing

In [10]:
# using the same DataFrame, index into into its first row
df.iloc[0,0]


0.12

In [11]:
df.loc['obs1', 'col_1']

0.12

# Mini-Challenge prep: data loading

## 1. How to load the `users` and `movies` portions of MovieLens

In [12]:
import pandas as pd

users = pd.read_table('data/ml-1m/users.dat',
                      sep='::', header=None,
                      names=['user_id', 'gender', 'age', 'occupation', 'zip'])

movies = pd.read_table('data/ml-1m/movies.dat',
                       sep='::', header=None,
                       names=['movie_id', 'title', 'genres'])

C:\Users\pamel\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
C:\Users\pamel\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


## 2. How to load the training and testing subsets

In [13]:
# subset version (hosted notebook)
movielens_train = pd.read_csv('data/movielens_train.csv', index_col=0, encoding='latin1')
movielens_test = pd.read_csv('data/movielens_test.csv', index_col=0, encoding='latin1')

In [14]:
movielens_train.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
593263,3562,3798,4,967332344,F,25,6,32812,What Lies Beneath (2000),Thriller,False
235597,1051,3793,4,974958593,F,25,0,60513,X-Men (2000),Action|Sci-Fi,False
219003,3727,2366,3,966309522,M,35,7,74401,King Kong (1933),Action|Adventure|Horror,False
685090,4666,1094,3,963843918,M,35,1,53704,"Crying Game, The (1992)",Drama|Romance|War,False
312377,3261,1095,4,968251750,M,45,20,87505,Glengarry Glen Ross (1992),Drama,False


In [15]:
movielens_test.head()

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693323,4653,2648,4,975532459,M,35,12,95051,Frankenstein (1931),Horror,False
24177,2259,1270,4,974591524,F,56,16,70503,Back to the Future (1985),Comedy|Sci-Fi,False
202202,3032,1378,5,970343147,M,25,0,47303,Young Guns (1988),Action|Comedy|Western,False
262003,3029,2289,4,972846393,M,18,4,92037,"Player, The (1992)",Comedy|Drama,False
777848,4186,2403,3,1017931262,M,25,7,33308,First Blood (1982),Action,False


# Mini-Challenge prep: evaluation functions

These are the two functions that you will need to test your `estimate` method.

In [16]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [17]:
def evaluate(estimate_f):
    """ RMSE-based predictive performance evaluation with pandas. """
    
    ids_to_estimate = zip(movielens_test.user_id, movielens_test.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in ids_to_estimate])
    real = movielens_test.rating.values
    return compute_rmse(estimated, real)

Test a dummy solution!

In [18]:
def my_estimate_func(user_id, movie_id):
    return 3.0

You can test for performance with the following line, which assumes that your function is called `my_estimate_func`:

In [19]:
print ('RMSE for my estimate function: %s' % evaluate(my_estimate_func))

RMSE for my estimate function: 1.23237195265


# Reco systems questions: Minimal reco engine v1.0

## 1. Simple collaborative filtering using mean ratings

In [20]:
# write an 'estimate' function that computes the mean rating of a particular user
def collab_mean(user_id, movie_id):
    
    item_condition=movielens_train.movie_id== movie_id
    return movielens_train.loc[item_condition, 'rating'].mean()
    
# try it out for a user_id, movie_id pair
collab_mean(4653, 2648)

4.0

# Mini-Challenge: first round
Implement an `estimate` function of your own using other similarity notions, eg.:

- collaborative filter based on age similarities
- collaborative filter based on occupation similarities
- content filter based on movie genre

In [89]:
movielens_train[movielens_train.movie_id==2648]

,user_id,movie_id,rating,timestamp,gender,age,occupation,zip,title,genres,for_testing
693112,1717,2648,5,974706281,F,50,6,30307,Frankenstein (1931),Horror,False
693286,4260,2648,3,965322649,M,25,16,59079,Frankenstein (1931),Horror,False


### Collaborative Filter based on Age Similarities

In [50]:
def collab_age(age, movie_id):
    
    item_condition=movielens_train.age==age
    item_condtion=movielens_train.loc[item_condition, 'movie_id']==movie_id
    return movielens_train.loc[item_condition, 'rating'].mean()
    
# try it out for a age_id, movie_id pair
collab_age(50, 2648)

3.704156479217604

### Collaborative Filter based on occupation Similarities

In [79]:
def collab_occupation(occupation, movie_id):
    
    item_condition=movielens_train.occupation==occupation
    item_condtion=movielens_train.loc[item_condition, 'movie_id']==movie_id
    return movielens_train.loc[item_condition, 'rating'].mean()
# try it out for a user_id, movie_id pair
collab_occupation(6, 2648)

3.533333333333333

### Collaborative Filter based on Genre Similaritoes

In [86]:
def collab_genre(genre, movie_id):
    
    item_condition=movielens_train.genres==genre
    item_condtion=movielens_train.loc[item_condition, 'movie_id']==movie_id
    return movielens_train.loc[item_condition, 'rating'].mean()
# try it out for a user_id, movie_id pair
collab_genre('Horror', 2648)

3.018633540372671

# Mini-Challenge: second round
Implement an `estimate` function of your own using other custom similarity notions, eg.:

- euclidean
- cosine

In [94]:
user_condition=movielens_train.user_id !=1717
movie_condition=movielens_train.movie_id == 2648
ratings_by_others=movielens_train.loc[user_condition & movie_condition]
ratings_by_others


    
means_by_zip=ratings_by_others.pivot_table('rating', index='movie_id', columns='zip')
means_by_zip
    

zip,59079
movie_id,
2648,3


In [90]:
def collab_zip(user_id, movie_id):
    """ Collaborative Filtering using an implicit sim(u,u) based on gender"""
    user_condition=movielens_train.user_id !=user_id
    movie_condition=movielens_train.movie_id == movie_id
    ratings_by_others=movielens_train.loc[user_condition & movie_condition]
    
    if ratings_by_others.empty:
        return 3.0
    
    means_by_zip=ratings_by_others.pivot_table('rating', index='movie_id', columns='zip')
    user_zip=user_info.ix[user_id, 'zip']
    if user_zip in means_by_zip.columns:
        return means_by_zip.ix[movie_id, user_zip]
    else:
        return means_by_zip.ix[movie_id].mean()
print (evaluate(collab_zip))

NameError: name 'user_info' is not defined